# Script to automatically make diacritical corrections in Hebrew text

## Defines letters and words

In [41]:
aleph_bet = ['א','ב','ג','ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם',
             'נ','ן','ס','ע','פ','ף','צ','ק','ר','ש','ת','װ','ױ','ײ','יִ',
             'ﬡ','ﬢ','ﬣ','ﬤ','ﬥ','ﬦ','ﬧ','ﬨ','שׁ','שׂ','שּׁ','שּׂ','אַ','אָ',
             'גּ','דּ','הּ','וּ','זּ','טּ','יּ','ךּ','כּ','לּ','מּ','נּ','סּ','ףּ',
             'פּ','צּ','שּ','תּ','וֹ','בֿ','כֿ','פֿ','ﭏ','בּ', 'קּ']

cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

vowel = ['ְ','ֱ','ֲ','ֳ','ִ','ֵ','ֶ','ַ','ָ','ֹ','ֺ','ֻ','ּ','ֽ','־','ֿ','ׁ','ׂ','ׄ','ׅ','ׇ']

letter_with_dagesh = ['שּׁ','שּׂ','גּ','דּ','זּ','טּ','יּ','כּ','לּ','מּ','נּ','סּ','פּ','צּ','שּ','תּ','בּ','קּ']
rafe = 'ֿ'
shva = vowel[0]
chataf_vowels = vowel[1:4]
short_vowels = [vowel[4],vowel[6],vowel[7],vowel[11],vowel[20]]
long_vowels = ['וֹ','וּ', vowel[5], vowel[8], aleph_bet[9], 'ֹ']
vowels_limited = vowel[0:4]+short_vowels+long_vowels
dagesh = 'ּ'
maqqaf = '־'
meteg = 'ֽ'
dot = '֯'

gutterals = ['א','ה','ח','ע','ר' ,'ﬡ','ﬣ','ﬧ']             

In [42]:
shem = 'יהוה'
shem_vowels = 'יְהֹוָה'
yy_vowels = aleph_bet[9]+vowel[0]+aleph_bet[9]+vowel[8]

kal = aleph_bet[52] + vowel[8] + aleph_bet[12]
khal = aleph_bet[10] + vowel[8] + aleph_bet[12]
kol_maqqaf = aleph_bet[52] + vowel[20] + aleph_bet[12] + '־'
khol_maqqaf = aleph_bet[10] + vowel[20] + aleph_bet[12] + '־'
kol_space = aleph_bet[52] + vowel[20] + aleph_bet[12] + ' '
khol_space = aleph_bet[10] + vowel[20] + aleph_bet[12] + ' '

et = 'אֶת'
ve_et = 'וְאֶת'
space_et_space = ' ' + 'אֶת' + ' '
space_ve_et_space = ' ' + 'וְאֶת' + ' '
et_maqqaf = ' ' + 'אֶת' + '־'
ve_et_maqqaf = ' ' + 'וְאֶת' + '־'

In [43]:
shva_exceptions = ['שְׁתֵּי','שְׁתָּיִם','שְׁתַּיִם','שְׁנִַים','שְׁנֵי','שְׁתֵּים','שְׁנֵים']
battim = 'בָּתִּים'
vattim = 'בָתִּים'
ana = 'אָנָּא'
anah = 'אָנָּה'
kamatz_katan_exceptions = [battim, vattim, ana, anah]

In [44]:
#the shem converter "eats" some special characters next to sheimot, this stops it from doing that
special_characters = ['{', '}','\\',]

## Strips vowels and cantelation marks from words

In [45]:
#removes trop and vowels
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [46]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant:
            no_cant_word = no_cant_word + letter
    return no_cant_word

## Converts Shem-Havaya to double-yud while preserving cantelation marks

In [47]:
def shem_converter(word):
    prefix = ''
    if word[0] != aleph_bet[9]:
        prefix = prefix + word[0]
        i = 1
        if word[1].isalpha() == False:
            prefix = prefix + word[1]
            if word[2].isalpha() == False:
                prefix = prefix + word[2]
    no_prefix_word = word[len(prefix):]

    if len(prefix) > 0:
        yud1 = aleph_bet[9]
    else:
        yud1 = aleph_bet[9]+vowel[0]

    yud2 = aleph_bet[9]+vowel[8]

    cant_marks = []

    #finds suffix
    suffix = ''
    i = -1
    while word[i] != aleph_bet[4]:
        if word[i] in special_characters:
            suffix = word[i] + suffix
        i = i-1
    
    for character in no_prefix_word:
        if character in cant:
            cant_marks.append(character)
    if len(cant_marks) == 0:
        new_shem = prefix + yud1 + yud2 + suffix
    elif len(cant_marks) == 1:
        new_shem = prefix + yud1 + yud2 + cant_marks[0] + suffix
    elif len(cant_marks) == 2:
        new_shem = prefix + yud1 + cant_marks[0] + yud2 + cant_marks[1] + suffix

    return new_shem

In [48]:
## Creates new paragraph with double-yud in place of Shem Havaya

In [49]:
#this needs to be fixed to stop eating special characters (i.e. brackets, \)

In [50]:
def convert_shem(paragraph):
    paragraph = str.replace(paragraph, '־', ' ־ ')
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        if shem in nonalpha_remover(word):
            double_yud = shem_converter(word)
            par_list[index] = double_yud

    new_par = ' '.join(par_list)
    new_par = str.replace(new_par, ' ־ ','־')
    new_par = str.replace(new_par, '׃', '׃')
    return new_par

In [51]:
## use https://he.wikisource.org/wiki/%D7%9E%D7%A7%D7%A8%D7%90 for testing texts

## Puts a kamatz katan and maqqaf in "kol"

In [52]:
#this script does not work on words with trope
#Since pesukim from wikisource have maqqafs and kamatz katans, it shouldn't make it less useful

In [53]:
def kamatz_exception(word):
    if word[-1] != aleph_bet[12]:
        return True
    elif aleph_bet[9] in word:
        return True
    else:
        return False
    #returns "true" if the word is a likely false positive for the word "kol"

In [54]:
def kol_kamatz_katan(paragraph):
    paragraph = str.replace(paragraph, '־', ' ־ ')
    par_list = paragraph.split()
    #Paragraph is now split into words
    for word in par_list:
        index = par_list.index(word)
        if (kal in word or khal in word) and kamatz_exception(word) == False:
            split_word = word.split(vowel[8])
            word = vowel[20].join(split_word)
            par_list[index] = word
            #If "k(h)al" appears in a word, change it to a kamatz katan
    new_par = ' '.join(par_list)
    new_par = str.replace(new_par, ' ־ ','־')
    #Put paragraph back together
    if kol_space in new_par: 
        new_par_split = new_par.split(kol_space)
        new_par = kol_maqqaf.join(new_par_split)
    if khol_space in new_par:
        new_par_split = new_par.split(khol_space)
        new_par = khol_maqqaf.join(new_par_split)
    #Previous if-statements swap a space following "kol" to a maqqaf
    new_par = str.replace(new_par, '׃', '׃')
    return new_par

## Puts a Maqqaf after "et" (when it has a segol)

In [15]:
#this script does not work on words with trope
#Since pesukim from wikisource have maqqafs, this shouldn't be a problem.

In [16]:
def et_fixer(paragraph):
    par_list = paragraph.split()
    if et in par_list:
        new_par_split = paragraph.split(space_et_space)
        new_paragraph = et_maqqaf.join(new_par_split)
        new_par_split = new_paragraph.split(space_ve_et_space)
        new_paragraph = ve_et_maqqaf.join(new_par_split)
    else:
        new_paragraph = paragraph
    return new_paragraph

## Puts a kamatz katan in common kamatz-katan words and situations

In [17]:
#this script does not work on words with trope
#Since pesukim from wikisource have kamatz katans, this shouldn't be too much of a problem.

In [18]:
chakhma = 'חָכְמָה'
chokhma = 'חׇכְמָה'
karban = 'קָרְבַּן'
korban = 'קׇרְבַּן'
kareinu = 'קָרְאֵֽנוּ'
koreinu = 'קׇרְאֵֽנוּ'
karbe_ = 'קָרְבְּ'
korbe_ = 'קׇרְבְּ'
kadshe_ = 'קָדְשְׁ'
kodshe_ = 'קׇדְשְׁ'
kadashim = 'קׇדָשִׁים'
kodashim = 'קׇדָשִׁים'

words_to_fix = [chakhma, karban, kareinu, karbe_, kadshe_, kadashim]
corrected_words = [chokhma, korban, koreinu, korbe_, kodshe_, kodashim]

In [19]:
def kamatz_katan_adder(word):    
    #prevents script from erroneously fixing false positives
    nt_word = trop_remover(word)
    for index in range(0,len(kamatz_katan_exceptions)):
        if kamatz_katan_exceptions[index] in nt_word:
            return word
    
    for index in range(1,len(word)-2):
        if word[index] == vowel[8]:
            i = index
            while word[i] not in aleph_bet:
                i = i+1
                if i>len(word)-1:
                    break
            if i<len(word)-1:
                next_consonant = word[i]
                if next_consonant in letter_with_dagesh:
                    word = word[:index]+vowel[20]+word[index:]
            
            i = index+1
            while word[i] not in vowels_limited:
                i = i+1
                if i>len(word)-1:
                    return word
            next_vowel = word[i]
            if next_vowel == chataf_vowels[2] and next_consonant in gutterals:
                word = word[:index]+vowel[20]+word[index:]
                return word
            #finds the vowel after the next one.  If both are shvas, the kamatz is katan.
            if next_vowel == shva and shva in word[index+1:]:
                if i<len(word)-1:
                    i = i+1
                    while i<len(word)-2 and word[i] not in vowels_limited:
                        i = i+1
                    sec_vowel = word[i]
                    if sec_vowel == shva:
                        word = word[:index]+vowel[20]+word[index:]

            #if there's a kamatz under the second-to-last consonant, and a meteg earlier in the word, the kamatz is katan
            if meteg in word and index == len(word)-2:
                #finds index of previous consonant.  Adds kamatz katan if meteg is before that.
                while word[i] not in aleph_bet:
                    i = i-1
                while i>0 and word[i] != meteg:
                    i = i-1
                if word[i] == meteg:
                    word = word[:index]+vowel[20]+word[index:]
    return word

In [20]:
def kamatz_katan(paragraph):
    #This goes through the list of common words with kamatz katan (besides "kol")
    #and corrects them if they are present.
    #This list can be added to as needed.
    for index in range(len(words_to_fix)):
        paragraph = paragraph.replace(words_to_fix[index],corrected_words[index])
    par_list = paragraph.split()
    #Calls the kamatz_katan_adder for each word in paragraph, if that word has a kamatz at all
    for index in range(len(par_list)-1):
        if vowel[8] in par_list[index]:
            par_list[index] = kamatz_katan_adder(par_list[index])
    paragraph = ' '.join(par_list)
    return paragraph  

In [21]:
test = 'וַיְדַבֵּ֥ר יְיָ֖ אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃ קַדֶּשׁ־לִ֨י כׇל־בְּֿכ֜וֹר פֶּ֤טֶר כׇּל־רֶ֙חֶם֙ בִּבְנֵ֣י יִשְׂרָאֵ֔ל בָּאָדָ֖ם וּבַבְּֿהֵמָ֑ה לִ֖י הֽוּא׃ וַיֹּ֨אמֶר מֹשֶׁ֜ה אֶל־הָעָ֗ם זָכ֞וֹר אֶת־הַיּ֤וֹם הַזֶּה֙ אֲשֶׁ֨ר יְֿצָאתֶ֤ם מִמִּצְרַ֙יִם֙ מִבֵּ֣ית עֲבָדִ֔ים כִּ֚י בְּֿחֹ֣זֶק יָ֔ד הוֹצִ֧יא יְיָ֛ אֶתְכֶ֖ם מִזֶּ֑ה וְֿלֹ֥א יֵאָכֵ֖ל חָמֵֽץ׃ ד הַיּ֖וֹם אַתֶּ֣ם יֹצְֿאִ֑ים בְּֿחֹ֖דֶשׁ הָאָבִֽיב׃ וְֿהָיָ֣ה כִֽי־יְֿֿבִיאֲךָ֣ יְיָ֡ אֶל־אֶ֣רֶץ הַֽ֠כְּֿנַעֲנִ֠י וְֿהַחִתִּ֨י וְֿהָאֱמֹרִ֜י וְֿהַחִוִּ֣י וְֿהַיְבוּסִ֗י אֲשֶׁ֨ר נִשְׁבַּ֤ע לַאֲבֹתֶ֙יךָ֙ לָ֣תֶת לָ֔ךְ אֶ֛רֶץ זָבַ֥ת חָלָ֖ב וּדְבָ֑שׁ וְֿעָבַדְתָּ֛ אֶת־הָעֲבֹדָ֥ה הַזֹּ֖את בַּחֹ֥דֶשׁ הַזֶּֽה׃ שִׁבְעַ֥ת יָמִ֖ים תֹּאכַ֣ל מַצֹּ֑ת וּבַיּוֹם֙ הַשְּֿׁבִיעִ֔י חַ֖ג לַייָ׃ מַצּוֹת֙ יֵֽאָכֵ֔ל אֵ֖ת שִׁבְעַ֣ת הַיָּמִ֑ים וְֿלֹֽא־יֵרָאֶ֨ה לְֿךָ֜ חָמֵ֗ץ וְֿלֹֽא־יֵרָאֶ֥ה לְֿךָ֛ שְֿׂאֹ֖ר בְּֿכׇל־גְּֿבֻלֶֽךָ׃ וְֿהִגַּדְתָּ֣ לְֿבִנְךָ֔ בַּיּ֥וֹם הַה֖וּא לֵאמֹ֑ר בַּעֲב֣וּר זֶ֗ה עָשָׂ֤ה יְיָ֙ לִ֔י בְּֿצֵאתִ֖י מִמִּצְרָֽיִם׃ וְֿהָיָה֩ לְֿךָ֨ לְֿא֜וֹת עַל־יָדְֿךָ֗ וּלְזִכָּרוֹן֙ בֵּ֣ין עֵינֶ֔יךָ לְֿמַ֗עַן תִּהְיֶ֛ה תּוֹרַ֥ת יְיָ֖ בְּֿפִ֑יךָ כִּ֚י בְּֿיָ֣ד חֲזָקָ֔ה הוֹצִֽאֲךָ֥ יְיָ֖ מִמִּצְרָֽיִם׃ וְֿשָׁמַרְתָּ֛ אֶת־הַחֻקָּ֥ה הַזֹּ֖את לְֿמוֹעֲדָ֑הּ מִיָּמִ֖ים יָמִֽימָה׃\\'

In [22]:
kamatz_katan(test)

'וַיְדַבֵּ֥ר יְיָ֖ אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃ קַדֶּשׁ־לִ֨י כׇל־בְּֿכ֜וֹר פֶּ֤טֶר כׇּל־רֶ֙חֶם֙ בִּבְנֵ֣י יִשְׂרָאֵ֔ל בָּאָדָ֖ם וּבַבְּֿהֵמָ֑ה לִ֖י הֽוּא׃ וַיֹּ֨אמֶר מֹשֶׁ֜ה אֶל־הָעָ֗ם זָכ֞וֹר אֶת־הַיּ֤וֹם הַזֶּה֙ אֲשֶׁ֨ר יְֿצָאתֶ֤ם מִמִּצְרַ֙יִם֙ מִבֵּ֣ית עֲבָדִ֔ים כִּ֚י בְּֿחֹ֣זֶק יָ֔ד הוֹצִ֧יא יְיָ֛ אֶתְכֶ֖ם מִזֶּ֑ה וְֿלֹ֥א יֵאָכֵ֖ל חָמֵֽץ׃ ד הַיּ֖וֹם אַתֶּ֣ם יֹצְֿאִ֑ים בְּֿחֹ֖דֶשׁ הָאָבִֽיב׃ וְֿהָיָ֣ה כִֽי־יְֿֿבִיאֲךָ֣ יְיָ֡ אֶל־אֶ֣רֶץ הַֽ֠כְּֿנַעֲנִ֠י וְֿהַחִתִּ֨י וְֿהָאֱמֹרִ֜י וְֿהַחִוִּ֣י וְֿהַיְבוּסִ֗י אֲשֶׁ֨ר נִשְׁבַּ֤ע לַאֲבֹתֶ֙יךָ֙ לָ֣תֶת לָ֔ךְ אֶ֛רֶץ זָבַ֥ת חָלָ֖ב וּדְבָ֑שׁ וְֿעָבַדְתָּ֛ אֶת־הָעֲבֹדָ֥ה הַזֹּ֖את בַּחֹ֥דֶשׁ הַזֶּֽה׃ שִׁבְעַ֥ת יָמִ֖ים תֹּאכַ֣ל מַצֹּ֑ת וּבַיּוֹם֙ הַשְּֿׁבִיעִ֔י חַ֖ג לַייָ׃ מַצּוֹת֙ יֵֽאָכֵ֔ל אֵ֖ת שִׁבְעַ֣ת הַיָּמִ֑ים וְֿלֹֽא־יֵרָאֶ֨ה לְֿךָ֜ חָמֵ֗ץ וְֿלֹֽא־יֵרָאֶ֥ה לְֿךָ֛ שְֿׂאֹ֖ר בְּֿכׇל־גְּֿבֻלֶֽךָ׃ וְֿהִגַּדְתָּ֣ לְֿבִנְךָ֔ בַּיּ֥וֹם הַה֖וּא לֵאמֹ֑ר בַּעֲב֣וּר זֶ֗ה עָשָׂ֤ה יְיָ֙ לִ֔י בְּֿצֵאתִ֖י מִמִּצְרָֽיִם׃ וְֿהָיָה֩ לְֿךָ֨ לְֿא֜וֹת עַל־יָדְֿךָ֗ 

In [23]:
word = 'לָ֔ךְ'

In [24]:
word[4]

'ְ'

## Marks shva na'

In [25]:
#Inserts a rafe to mark a shva na'
def na_marker(paragraph,index):
    if paragraph[index-1] == rafe or paragraph[index+1] == rafe:
        return paragraph
        #does nothing, if the shva is already marked with a rafe
    else:
        par_start = paragraph[:index]
        par_end = paragraph[index:]
        new_par = par_start+rafe+par_end
        return new_par
        #adds a rafe over the input letter

In [26]:
#Determines what shvas are na' in a word, and calls the program to mark them
def shva_na_function(word):
    nt_word = trop_remover(word)
    #this skips instances of shem hashem
    if shem_vowels in nt_word or yy_vowels in nt_word:
        return word
    if nt_word in shva_exceptions:
        return word
    for index in range(1,len(word)-2):
        i = index
        if word[index] != shva:
            continue
            #this doesn't bother with the loop if the letter isn't a shva
            
        while word[i] not in aleph_bet:
            i = i-1
        previous_consonant = word[i]
        #if the previous consonant is the beginning of the word, the shva is na'
        if i == 0:
            word = na_marker(word,index)
            continue
        if word[i-1] == maqqaf:
            word = na_marker(word,index)
        #if the previous consonant has a dagesh, the shva must be na'
        if previous_consonant in letter_with_dagesh:
            word = na_marker(word,index)
            continue
        if word[i+1] == dagesh or word[index+1] == dagesh:
            word = na_marker(word,index)
            continue
            
        #determines the next consonant
        i = index + 1
        while word[i] not in aleph_bet and i<len(word)-1:
            i = i+1
        next_consonant = word[i]
        if next_consonant == previous_consonant:
            word = na_marker(word,index)
            continue

        #determines the previous vowel
        i = index-2
        while word[i] not in vowels_limited and i>0:
            i = i-1
        if i == 0:
            return word
        else:
            previous_vowel = word[i]
            #if the previous vowel is a shva, the current shva is na'
            if previous_vowel == shva:
                word = na_marker(word,index)
            if previous_vowel in long_vowels:
                word = na_marker(word,index)    
        
        #a shva following a long vowel is na', unless the long vowel is word-initial shuruk
   

    #determine how to mark סקינמלוי letters
    return word

In [27]:
#Calls the shva_na_function for each word in the input paragraph
def shva_na_converter(paragraph):
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        par_list[index] = shva_na_function(word)
    new_par = ' '.join(par_list)
    return new_par

## Changes shva marking character

In [55]:
#this program allows the changing of all rafes used to mark shva into another character
#in this case a circle above the letter
#this could be adapted to change a text that uses the dot to using something else
def shva_na_mark_changer(paragraph):
    new_paragraph = paragraph.replace(rafe,dot)
    return new_paragraph
    #return new_paragraph

## Mistake catcher

In [74]:
#Catches typographical issues lingering around in here
def mistake_catcher(paragraph):
    dot2 = dot + dot
    rafe2 = rafe + rafe
    _2kamatz1 = vowel[8] + vowel[20]
    _2kamatz2 = vowel[20] + vowel[8]
    while dot2 in paragraph or rafe2 in paragraph or _2kamatz1 in paragraph or _2kamatz2 in paragraph:
        paragraph = paragraph.replace(dot2,dot)
        paragraph = paragraph.replace(rafe2,rafe)
        paragraph = paragraph.replace(_2kamatz1,vowel[20])
        paragraph = paragraph.replace(_2kamatz2,vowel[20])
    return paragraph

In [75]:
test = 'וְֿצׇׇׇהֳרָֽיִם'
_2kamatz1 = vowel[8] + vowel[20]
_2kamatz2 = vowel[20] + vowel[8]
if _2kamatz1 in test:
    print('1')
if _2kamatz2 in test:
    print('2')

In [76]:
test_fixed = test.replace(_2kamatz1,vowel[20])
test_fixed

'וְֿצׇׇׇהֳרָֽיִם'

In [77]:
test2 = 'יְ֯֯בִאֲךָ֤'

In [33]:
if _2kamatz2 in test or _2kamatz1 in test:
    print('true')

In [67]:
mistake_catcher(test2)

script got here


'יְ֯בִאֲךָ֤'

## Runs Paragraph through all converters

In [78]:
def text_converter(paragraph):
    #Comment out components of the script you don't want to run
    paragraph = convert_shem(paragraph)
    #paragraph = kol_kamatz_katan(paragraph)
    #paragraph = et_fixer(paragraph)
    #paragraph = kamatz_katan(paragraph)
    #paragraph = shva_na_converter(paragraph)
    paragraph = shva_na_mark_changer(paragraph)
    paragraph = mistake_catcher(paragraph)
    return paragraph

## Reading file from disk

In [79]:
input_file = 'siddur.tex'

In [80]:
with open(input_file, 'r', encoding='utf-8') as infile:
    lines = list(infile.readlines())

In [81]:
new_lines = []
line_number = 1
for line in lines:
    new_lines.append(text_converter(line))
    #for troubleshooting--identifies the last line where the script ran
    #print('line number', line_number, 'outputted successfully')
    line_number = line_number+1

In [82]:
output_file = 'siddur_converted.tex'

In [83]:
with open(output_file, 'w', encoding='utf-8') as outfile:
    for line in new_lines:
        if line != '':
            if line[-1] != '\n':
                outfile.write(line + '\n')
            else:
                outfile.write(line)
        else:
            outfile.write('\n')